https://docs.google.com/document/d/1qJaeuC4Fddou_Ts2Lea0UHjrzgy8aSzY6ACCvUwuTe4/edit

In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

In [100]:
#load and preprocess countries data
df_countries = pd.read_csv('countries_all.csv')
filtered_df_countries = (df_countries[df_countries['Time'] == 2020]
                         .rename(columns={'Country Name' : 'country', 
                                          'Time' : 'year', 
                                          'GDP per capita, PPP (current international $) [NY.GDP.PCAP.PP.CD]' : 'gdp', 
                                          'CO2 emissions (metric tons per capita) [EN.ATM.CO2E.PC]' : 'co2'})
                         [['country', 'year', 'gdp', 'co2']])

#load and preprocess esg data
esg = pd.read_csv('esg.csv')
esg.replace('..', np.nan, inplace=True)
esg.dropna(inplace=True)

#load and preprocess culture data
cul = pd.read_csv('culture.csv')
cul.replace('#NULL!', np.nan, inplace=True)
cul.dropna(inplace=True)

#merge all dataframes
dfs = [filtered_df_countries, esg, cul]
combined_df = reduce(lambda left,right: pd.merge(left,right,on='country'), dfs)

#sort values and reset index in-place
combined_df.sort_values('country', inplace=True)
combined_df.reset_index(drop=True, inplace=True)

#output
combined_df.head(5)


,country,year,gdp,co2,mean_of_esg,pdi,idv,mas,uai,ltowvs,ivr
0,Argentina,2020.0,20797.7259531997,3.4056175404138,0.552,49,46,56,86,20,62
1,Australia,2020.0,53216.4685369712,14.7761369066259,0.560,38,90,61,51,21,71
2,Austria,2020.0,57235.5458694937,6.63264573733546,0.613,11,55,79,70,60,63
3,Bangladesh,2020.0,5904.56736266229,0.510647559277094,0.532,80,20,55,60,47,20
4,Belgium,2020.0,54679.1418047701,7.39813065774681,0.586,65,75,54,94,82,57
